In [17]:
import requests
import os
import math
import json
import pprint as pp
import cv2
import numpy as np
import base64
from dotenv import load_dotenv
 
load_dotenv()

def read_input():
    # read input file
    with open('inputs.json', 'r') as inputs:
        data=inputs.read()

    # parse input file
    input_json = json.loads(data)

    # seting main parameters
    input_json['API_KEY'] = os.environ["API_KEY_MAPS"]
    input_json['API_ENDPOINT_URL'] = "https://maps.googleapis.com/maps/api/staticmap"
    input_json['IMAGE_DIR'] = "C:/Users/fonss/Documents/Programacion/GIT/sid_image_extractor/maps_images/"
    return input_json

input_json = read_input()

In [68]:
# Generate the name of the auxiliary images depending of the input data
def get_sub_image_name(data, input_json):

    name =  data['name']
    zoom = str(input_json['image_params']['zoom'])
    size = str(input_json['image_params']['size'])
    pos = str(data['center']['pos'])

    image_ref = name + '_' + zoom + '_' + size + '_' + pos + '.png'
    return image_ref

# Generate the name of the global image depending of the input data
def get_image_name(data, input_json):

    name =  data['name']
    grid_size = str(data['grid_size'])
    zoom = str(input_json['image_params']['zoom'])
    size = str(input_json['image_params']['size'])

    image_ref = name + '_' + grid_size + '_' + zoom + '_' + size + '.png'
    return image_ref

# Generate the folder name for the data input 
def get_sub_folder_ref(data, input_json):

    name =  data['name']
    grid_size = str(data['grid_size'])
    zoom = str(input_json['image_params']['zoom'])
    size = str(input_json['image_params']['size'])

    folder_ref = input_json['IMAGE_DIR'] + name + '_' + grid_size + '_' + zoom + '_' + size + '/'
    return folder_ref

# Create the folder and saves the auxiliar image 
def save_sub_image(data, input_json, img):

    folder_ref = get_sub_folder_ref( data , input_json )

    if not os.path.exists(folder_ref):
        os.makedirs(folder_ref)

    image_name = get_sub_image_name( data , input_json )

    image_ref = folder_ref + image_name

    # save image in directory
    with open(image_ref, 'wb') as f:
        f.write(img)

# Create the folder and saves the global image 
def save_image(data, input_json, img):

    if not os.path.exists(input_json['IMAGE_DIR']):
        os.makedirs(input_json['IMAGE_DIR'])

    image_name = get_image_name( data , input_json )

    image_ref = input_json['IMAGE_DIR'] + image_name

    # save image in directory
    cv2.imwrite(image_ref, img)
    

# makes the request of the images and returns it
def get_image(data, input_json):

    # create the params object for the request
    params = {
        'center': str(data['center']['lat']) + ',' + str(data['center']['lon']),
        'zoom': input_json['image_params']['zoom'],
        'maptype': input_json['image_params']['maptype'],
        'size': str(input_json['image_params']['size']) + 'x' + str(input_json['image_params']['size']),
        'scale':input_json['image_params']['scale'],
        'key': input_json['API_KEY'],
    }

    # make the request to the Maps Static API
    res = requests.get(url = input_json['API_ENDPOINT_URL'], params = params)

    # return the image
    return res.content

def get_m_per_px(data, input_json):
    m_per_px_lat = 156543.03392 * math.cos(data['center']['lat'] * math.pi / 180) / math.pow(2, input_json['image_params']['zoom'])
    m_per_px_lon = 156543.03392 * math.cos(data['center']['lon'] * math.pi / 180) / math.pow(2, input_json['image_params']['zoom'])
    return m_per_px_lat, m_per_px_lon

def get_deg_per_m(data):
    err = 8.6/100 # 8.6% error
    deg_per_m_lat = 1 / 111320 * (1-err)
    deg_per_m_lon = -1 / (40075000 * math.cos(data['center']['lat']) / 360) * (1-err)
    return deg_per_m_lat, deg_per_m_lon


def get_grid(data, input_json):
    m_per_px_lat , m_per_px_lon = get_m_per_px(data,input_json)
    deg_per_m_lat , deg_per_m_lon = get_deg_per_m(data)

    overlap = 0
    if input_json['image_params']['overlap']:
        overlap = 23

    img_size_deg_lat = m_per_px_lat * deg_per_m_lat * (input_json['image_params']['size']-overlap)  # m/px * deg/m * px = deg
    img_size_deg_lon = m_per_px_lon * deg_per_m_lon * (input_json['image_params']['size']-overlap)  # m/px * deg/m * px = deg


    if data['grid_size'] == 1:
        return [data['center']]
    
    # create the grid depending for odd numbers 
    elif data['grid_size'] % 2 != 0 : # 3x3 5x5 7x7 ....
        n = data['grid_size']

        centers = []
        lats = []
        lons = []

        # create the first center point top left 
        lat_0 = data['center']['lat'] + (n-1) * (img_size_deg_lat / 2)
        lon_0 = data['center']['lon'] - (n-1) * (img_size_deg_lon / 2)

        # adds the following point to the list depending on the image size in deg
        for a in range(n):
            lat = lat_0 - img_size_deg_lat * a
            lon = lon_0 + img_size_deg_lon * a
            lats.append(lat)
            lons.append(lon)

        # joins the lat and lon position into an object with its coords position ex: 0-0 for the first one
        for i in range(len(lats)):
            lat = lats[i]
            for j in range(len(lons)):
                lon = lons[j]
                centers.append({
                    'pos': str(i) + "-" + str(j),
                    'lat': lat,
                    'lon': lon,
                })
        return centers

    # create the grid depending for even numbers 
    elif data['grid_size'] % 2 == 0: # 2x2 4x4 6x6

        n = data['grid_size']

        centers = []
        lats = []
        lons = []
        
        # create the first center point top left 
        lat_0 = data['center']['lat'] + (n-1) * ( img_size_deg_lat / 2 )
        lon_0 = data['center']['lon'] - (n-1) * ( img_size_deg_lon / 2 )

        # adds the following point to the list depending on the image size in deg
        for a in range(n):
            lat = lat_0 - img_size_deg_lat * a
            lon = lon_0 + img_size_deg_lon * a
            lats.append(lat)
            lons.append(lon)

        # joins the lat and lon position into an object with its coords position ex: 0-0 for the first one
        for i in range(len(lats)):
            lat = lats[i]
            for j in range(len(lons)):
                lon = lons[j]
                centers.append({
                    'pos': str(i) + "-" + str(j),
                    'lat': lat,
                    'lon': lon,
                })
        return centers
    else:
        return  []


def load_sub_image(data , input_json):
    ref = get_sub_folder_ref(data, input_json) + get_sub_image_name(data,input_json )
    img = cv2.imread(ref)
    return img

def crop_images(images, input_json):
    images_crop = []
    for img in images:
        crop = input_json['image_params']['size']-43
        images_crop.append(img[:crop,:])
    return images_crop

def join_images(images, grid , data , input_json):

    horizontals_group = []
    counter = 0

    images_croped = crop_images(images, input_json)

    print()
    
    for i in range(data['grid_size']):
        line = []

        for j in range(data['grid_size']):
            line.append(images_croped[counter])
            counter += 1

        line_img = cv2.hconcat(line)
        horizontals_group.append(line_img)
    
    global_img = cv2.vconcat(horizontals_group)
    return global_img


In [72]:
# Import the imput data from the JSON file
input_json = read_input()

for data in input_json['data']:
    # create the grid for depending on each input item
    grid = get_grid( data , input_json )

    images = []

    for point in grid:
        # rewrites the location of the center for each data point in the grid
        data['center']['lat'] = point['lat']
        data['center']['lon'] = point['lon']    
        data['center']['pos'] = point['pos']

        # downloads the image from the Maps Static API
        # img_aux = get_image(data, input_json)    # UNCOMMENT TO GET THE IMAGES

        # saves temporaly the auxiliary image 
        # save_sub_image(data, input_json, img_aux)

        # load the image as a matrix
        img = load_sub_image(data , input_json)
        images.append(img)
   
    global_img = join_images(images, grid , data , input_json)

    save_image(data, input_json, global_img)


In [ ]:
### Save Image Data in BigQuery

In [69]:

import pandas as pd
import pandas_gbq

# ___MAPS IMAGES___ 

def saveImages(images_df):
    destination_table = 'osm_data.maps_images'
    pandas_gbq.to_gbq(images_df, project_id='sidhouses', destination_table=destination_table , if_exists='replace')

def loadImages():
    df = pandas_gbq.read_gbq('SELECT * FROM sidhouses.osm_data.maps_images')
    return df


In [73]:

image_headers = ['img_name',
'name','input_lat',
'input_lon','input_size',
'grid_size',
'zoom',
'lat_size',
'lon_size',
'lat_0',
'lon_0',
'm_per_px_lat',
'm_per_px_lon',
'deg_per_m_lat',
'deg_per_m_lon', 
'deg_per_px_lat', 
'deg_per_px_lon'
]

input_json = read_input()
new_image_data = []

for data in input_json['data']:

    img_name = data['name'] + '_' + str(data['grid_size']) + '_' + str(input_json['image_params']['zoom']) + '_' + str(input_json['image_params']['size']) + '.png'

    grid = get_grid( data , input_json )

    m_per_px_lat , m_per_px_lon = get_m_per_px(data,input_json)
    deg_per_m_lat , deg_per_m_lon = get_deg_per_m(data)

    lat_0 = grid[0]['lat'] + input_json['image_params']['size']/2 * m_per_px_lat * deg_per_m_lat
    lon_0 = grid[0]['lon'] - input_json['image_params']['size']/2 * m_per_px_lon * deg_per_m_lon

    lat_size = input_json['image_params']['size'] * data['grid_size'] - data['grid_size'] * 43
    lon_size = input_json['image_params']['size'] * data['grid_size'] 
 
    ele_row = [ 
               img_name,                           # img_name
               data['name'],                       # name
               data['center']['lat'],              # input_lat
               data['center']['lon'],              # input_lon
               input_json['image_params']['size'], # input_size
               data['grid_size'],                  # grid_size
               input_json['image_params']['zoom'], # zoom
               lat_size,                           # lat_size
               lon_size,                           # lon_size
               lat_0,                              # lat_0
               lon_0,                              # lon_0
               m_per_px_lat,                       # m_per_px_lat
               m_per_px_lon,                       # m_per_px_lon
               deg_per_m_lat,                      # deg_per_m_lat
               deg_per_m_lon,                      # deg_per_m_lon
               m_per_px_lat * deg_per_m_lat,       # deg_per_px_lat
               m_per_px_lon * deg_per_m_lon,       # deg_per_px_lon
               ]

    new_image_data.append(ele_row)
    
# Create Dataframe from data
new_image_df = pd.DataFrame(new_image_data,columns=image_headers)
new_image_df.head()

,img_name,name,input_lat,input_lon,input_size,grid_size,zoom,lat_size,lon_size,lat_0,lon_0,m_per_px_lat,m_per_px_lon,deg_per_m_lat,deg_per_m_lon,deg_per_px_lat,deg_per_px_lon
0,Monnon_5_19_500.png,Monnon,9.842513,3.178864,500,5,19,2285,2500,9.845532,3.175517,0.294187,0.298123,0.000008,0.000009,0.000002,0.000003
1,Sansi Gando_5_19_500.png,Sansi Gando,9.947919,3.418565,500,5,19,2285,2500,9.950937,3.415034,0.294093,0.298051,0.000008,0.000009,0.000002,0.000003
2,Besen Gourou_5_19_500.png,Besen Gourou,9.905893,3.350370,500,5,19,2285,2500,9.908911,3.346920,0.294131,0.298072,0.000008,0.000009,0.000002,0.000003
3,Barkedje_5_19_500.png,Barkedje,10.025566,3.287887,500,5,19,2285,2500,10.028584,3.284179,0.294023,0.298091,0.000008,0.000010,0.000002,0.000003
4,Boudal_5_19_500.png,Boudal,9.866730,3.172789,500,5,19,2285,2500,9.869749,3.169404,0.294166,0.298124,0.000008,0.000009,0.000002,0.000003


In [ ]:
# Load images
images_df = loadImages()
images_df.shape

In [ ]:
def isImgalready(df, img_name):
  for index, row in df.iterrows():
    if row['img_name'] == img_name:
      return True
  return False

# Update missing values in dataframe from spreadsheet
for index, new_row in new_image_df.iterrows():
  if not isImgalready(images_df,new_row['img_name']) :
    images_df = images_df.append(new_row)

images_df.shape

In [74]:
# save Image df
saveImages(new_image_df)

NameError: name &#39;images_df&#39; is not defined